In [2]:
# Dependencies
import pandas as pd
import requests
import json
import pprint
import numpy as np
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps
from us import states

# Census & gmaps API Keys
from config import (api_key, gkey)
c = Census(api_key, year=2013)

# Configure gmaps
gmaps.configure(api_key=gkey)

In [3]:
#Zip Codes
file = 'zip_borough.csv'

#ZIPCODE LIST FOR FILTERING
filedf = pd.read_csv(file)
ziplst = filedf['zip'].tolist()
filedf.head()

,zip,borough
0,10001,Manhattan
1,10002,Manhattan
2,10003,Manhattan
3,10004,Manhattan
4,10005,Manhattan


In [4]:
#Zillow home value csv
homeValFile = 'zillow_home_values.csv'
homeVal_df = pd.read_csv(homeValFile, encoding='latin-1')
homeVal_df = homeVal_df.rename(columns={"RegionName": "Zipcode"})
homeVal_df = pd.DataFrame(homeVal_df)
homeVal_df.head()

,RegionID,Zipcode,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,168300.0,168400.0,168400.0,...,1072100,1066500,1054600,1032700,1017300,1011200,991800,974900,970400,969800
1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,160000.0,161000.0,161700.0,...,350700,352700,353400,353500,353800,353400,351600,348700,346500,346100
2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,353100.0,353700.0,354200.0,...,1391500,1383600,1378200,1373300,1373500,1366200,1359900,1370300,1397800,1412700
3,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,210400.0,212200.0,212200.0,...,336700,335900,336000,335600,334200,333400,333500,333500,331800,330100
4,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,5,194400.0,196100.0,197300.0,...,429900,431500,432700,433200,433000,431700,429700,427300,424300,422100


In [6]:
#convert into INT to allow Merge
homeVal_dff = homeVal_df.astype({'Zipcode':'int64'})

#filter for our NYC zipcodes
homeVal_df2 = homeVal_dff['Zipcode'].isin(ziplst)
homeVal_df = homeVal_dff[homeVal_df2]
homeVal_df = pd.DataFrame(homeVal_df)

homeVal_df

,RegionID,Zipcode,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,168300.0,168400.0,168400.0,...,1072100,1066500,1054600,1032700,1017300,1011200,991800,974900,970400,969800
2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,353100.0,353700.0,354200.0,...,1391500,1383600,1378200,1373300,1373500,1366200,1359900,1370300,1397800,1412700
8,61616,10002,New York,NY,New York-Newark-Jersey City,New York County,9,NaN,NaN,NaN,...,980800,973300,966800,955800,950100,943200,927800,914000,903500,894700
9,61807,10467,New York,NY,New York-Newark-Jersey City,Bronx County,10,140900.0,140900.0,140900.0,...,420900,421300,421700,423100,426200,429200,430200,430500,430700,430300
10,62037,11226,New York,NY,New York-Newark-Jersey City,Kings County,11,153000.0,153000.0,152900.0,...,828500,831200,832700,832300,828900,825600,824700,823700,820200,820200
13,61630,10016,New York,NY,New York-Newark-Jersey City,New York County,14,243700.0,242700.0,241600.0,...,1026600,1020800,1010400,999500,995200,993400,985600,976300,970200,966100
17,62045,11235,New York,NY,New York-Newark-Jersey City,Kings County,18,164100.0,164900.0,165900.0,...,713700,716400,717000,715100,718600,726100,734600,738200,738000,734600
18,62087,11375,New York,NY,New York-Newark-Jersey City,Queens County,19,229500.0,229500.0,230000.0,...,972600,985500,994400,995800,997900,1003700,1009500,1008700,1000300,990700
22,61643,10029,New York,NY,New York-Newark-Jersey City,New York County,23,188100.0,189800.0,191500.0,...,795300,794500,788300,783200,783600,784700,780300,769400,758800,752200
23,61623,10009,New York,NY,New York-Newark-Jersey City,New York County,24,NaN,NaN,NaN,...,1145600,1147700,1147800,1126400,1101600,1091900,1080600,1060200,1043700,1034300


In [10]:
# Median home value is given monthly, we've mainly dealt with yearly data so I averaged the months for each year

#create a work data frame with the months of 1996 and dropped missing values
yearData = pd.DataFrame(homeVal_df.iloc[:,7:16])
yearData = yearData.dropna()

#create clean data frame with our NY zipcodes as the first column
medCleanData = pd.DataFrame(homeVal_df['Zipcode'])

#add a column with the averages from 1996 (only 9 months given)
medCleanData['1996 Median Home Value'] = (yearData['1996-04'] + yearData['1996-05'] + yearData['1996-06'] +
                     yearData['1996-07'] + yearData['1996-08'] + yearData['1996-09'] + 
                     yearData['1996-10'] + yearData['1996-11'] + yearData['1996-12'])/9 
medCleanData['1996 Median Home Value'] = medCleanData['1996 Median Home Value'].round()

#loop through years 1997-2018 and complete the same process
yr = 1997
i = 16
while yr < 2019:
    yearData = pd.DataFrame(homeVal_df.iloc[:, i:(i+12)])
    yearData = yearData.dropna()
    medCleanData[str(yr)+' Median Home Value'] = (yearData[str(yr)+'-01'] + yearData[str(yr)+'-02'] + yearData[str(yr)+'-03'] + 
                          yearData[str(yr)+'-04'] + yearData[str(yr)+'-05'] + yearData[str(yr)+'-06'] + 
                          yearData[str(yr)+'-07'] + yearData[str(yr)+'-08'] + yearData[str(yr)+'-09'] + 
                          yearData[str(yr)+'-10'] + yearData[str(yr)+'-11'] + yearData[str(yr)+'-12'])/12
    medCleanData[str(yr)+' Median Home Value'] = medCleanData[str(yr)+' Median Home Value'].round()
    i = i + 12
    yr = yr + 1
    
#do same for year 2019 (only 7 months given)
yearData = pd.DataFrame(homeVal_df.iloc[:, i:(i+7)])
yearData = yearData.dropna()
medCleanData[str(yr)+' Median Home Value'] = (yearData[str(yr)+'-01'] + yearData[str(yr)+'-02'] + 
                      yearData[str(yr)+'-03'] + yearData[str(yr)+'-04'] + 
                      yearData[str(yr)+'-05'] + yearData[str(yr)+'-06'] + 
                      yearData[str(yr)+'-07'])/7
medCleanData[str(yr)+' Median Home Value'] = medCleanData[str(yr)+' Median Home Value'].round()


medCleanData.head()


,Zipcode,1996 Median Home Value,1997 Median Home Value,1998 Median Home Value,1999 Median Home Value,2000 Median Home Value,2001 Median Home Value,2002 Median Home Value,2003 Median Home Value,2004 Median Home Value,...,2010 Median Home Value,2011 Median Home Value,2012 Median Home Value,2013 Median Home Value,2014 Median Home Value,2015 Median Home Value,2016 Median Home Value,2017 Median Home Value,2018 Median Home Value,2019 Median Home Value
0,10025,169022.0,181767.0,210567.0,266700.0,355642.0,414375.0,415192.0,484758.0,577858.0,...,767983.0,740925.0,758875.0,869350.0,940908.0,1017367.0,1110608.0,1095017.0,1101108.0,995443.0
2,10023,355322.0,379225.0,434183.0,478775.0,563167.0,654467.0,678167.0,749042.0,830892.0,...,1007850.0,1163883.0,1229458.0,1323975.0,1413975.0,1480550.0,1595358.0,1605383.0,1460642.0,1379100.0
8,10002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,473925.0,...,641658.0,678158.0,718558.0,732975.0,793742.0,825633.0,885350.0,975483.0,988367.0,927014.0
9,10467,140722.0,142117.0,147367.0,157225.0,172733.0,191217.0,214867.0,234717.0,269517.0,...,300417.0,291392.0,270792.0,289508.0,310083.0,322825.0,344808.0,369408.0,408967.0,428600.0
10,11226,153433.0,158317.0,167650.0,178633.0,195575.0,216442.0,239417.0,283850.0,317325.0,...,394200.0,405758.0,442675.0,438208.0,487908.0,577958.0,624725.0,673608.0,800092.0,825086.0


In [12]:
change = medCleanData[['Zipcode', '2019 Median Home Value', '2018 Median Home Value']]
yr = 2018
change

,Zipcode,2019 Median Home Value,2018 Median Home Value
0,10025,995443.0,1101108.0
2,10023,1379100.0,1460642.0
8,10002,927014.0,988367.0
9,10467,428600.0,408967.0
10,11226,825086.0,800092.0
13,10016,983757.0,1028058.0
17,11235,729314.0,719642.0
18,11375,1000943.0,973358.0
22,10029,773171.0,789067.0
23,10009,1076957.0,1151383.0


In [23]:
#put individual year data frames into a list
yr = 1996
ZdataList = []
while yr < 2020:
    yearData = medCleanData[['Zipcode', str(yr)+' Median Home Value']]
    ZdataList.append(yearData)
    yr = yr + 1
ZdataList[23].head()   

,Zipcode,2019 Median Home Value
0,10025,995443.0
2,10023,1379100.0
8,10002,927014.0
9,10467,428600.0
10,11226,825086.0


In [7]:
#Zillow rent price csv
rentFile = 'zillow_rental_price.csv'
rent_df = pd.read_csv(rentFile, encoding='latin-1')
rent_df = rent_df.rename(columns={"RegionName": "Zipcode"})
rent_df = pd.DataFrame(rent_df)

#convert into INT to allow Merge
rent_dff = rent_df.astype({'Zipcode':'int64'})

#filter for our NYC zipcodes
rent_df2 = rent_dff['Zipcode'].isin(ziplst)
rent_df = rent_dff[rent_df2]
rent_df = pd.DataFrame(rent_df)

rent_df.head()

,Zipcode,City,State,Metro,CountyName,SizeRank,2010-02,2010-03,2010-04,2010-05,...,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07
0,10025,New York,NY,New York-Newark-Jersey City,New York County,1,NaN,NaN,NaN,NaN,...,3392.5,3400.0,3492.5,3400.0,3400.0,3402.5,3380.0,3500.0,3470.0,3402.5
2,10023,New York,NY,New York-Newark-Jersey City,New York County,3,NaN,NaN,3425.0,3250.0,...,3792.5,3750.0,3750.0,3750.0,3600.0,3500.0,3562.5,3695.0,3600.0,3587.5
8,10002,New York,NY,New York-Newark-Jersey City,New York County,9,NaN,NaN,NaN,NaN,...,3125.0,3125.0,3195.0,3050.0,3112.5,3100.0,3000.0,3250.0,3297.5,3195.0
9,10467,New York,NY,New York-Newark-Jersey City,Bronx County,10,NaN,NaN,NaN,NaN,...,1500.0,1400.0,1400.0,1500.0,1500.0,1550.0,1500.0,1500.0,1500.0,1487.5
10,11226,New York,NY,New York-Newark-Jersey City,Kings County,11,NaN,NaN,NaN,NaN,...,1750.0,1700.0,1800.0,1700.0,1687.5,1650.0,1675.0,1650.0,1600.0,1650.0


In [8]:
# Median rental is given monthly, we've mainly dealt with yearly data so I averaged the months for each year


#create clean data frame with our NY zipcodes as the first column
rentCleanData = pd.DataFrame(homeVal_df['Zipcode'])

#loop through years 2011-2018 and computed the average median rental price for each year
yr = 2011
i = 17
while yr < 2019:
    yearData = pd.DataFrame(rent_df.iloc[:, i:(i+12)])
    yearData = yearData.dropna()
    rentCleanData[str(yr)+' Median Rental Price'] = (yearData[str(yr)+'-01'] + yearData[str(yr)+'-02'] + yearData[str(yr)+'-03'] + 
                          yearData[str(yr)+'-04'] + yearData[str(yr)+'-05'] + yearData[str(yr)+'-06'] + 
                          yearData[str(yr)+'-07'] + yearData[str(yr)+'-08'] + yearData[str(yr)+'-09'] + 
                          yearData[str(yr)+'-10'] + yearData[str(yr)+'-11'] + yearData[str(yr)+'-12'])/12
    rentCleanData[str(yr)+' Median Rental Price'] = rentCleanData[str(yr)+' Median Rental Price'].round()
    i = i + 12
    yr = yr + 1
    
#do same for year 2019 (only 7 months given)
yearData = pd.DataFrame(rent_df.iloc[:, i:(i+7)])
yearData = yearData.dropna()
rentCleanData[str(yr)+' Median Rental Price'] = (yearData[str(yr)+'-01'] + yearData[str(yr)+'-02'] + 
                                              yearData[str(yr)+'-03'] + yearData[str(yr)+'-04'] + 
                                              yearData[str(yr)+'-05'] + yearData[str(yr)+'-06'] + 
                                              yearData[str(yr)+'-07'])/7
rentCleanData[str(yr)+' Median Rental Price'] = rentCleanData[str(yr)+' Median Rental Price'].round()

rentCleanData.head()

,Zipcode,2011 Median Rental Price,2012 Median Rental Price,2013 Median Rental Price,2014 Median Rental Price,2015 Median Rental Price,2016 Median Rental Price,2017 Median Rental Price,2018 Median Rental Price,2019 Median Rental Price
0,10025,2916.0,3045.0,3260.0,3389.0,3437.0,3474.0,3426.0,3402.0,3422.0
2,10023,3492.0,3464.0,3418.0,3727.0,3767.0,3659.0,3660.0,3611.0,3614.0
8,10002,NaN,2753.0,2878.0,3136.0,3153.0,3039.0,3110.0,3180.0,3144.0
9,10467,NaN,NaN,NaN,NaN,NaN,NaN,1268.0,1419.0,1505.0
10,11226,NaN,NaN,NaN,NaN,NaN,NaN,1591.0,1691.0,1659.0


In [28]:
#adding the renal column to each of our individual year data frames
i = 0
yr = 1996
ZZdataList = []
#no data for 1996-2010
while i < 15:
    ZdataList[i][str(yr)+' Median Rental Price'] = np.nan
    yearData = pd.DataFrame(ZdataList[i])
    ZZdataList.append(yearData)
    i = i + 1
    yr = yr + 1
#data from our rentCleanData table for 2011-2019   
while i < 24:
    ZdataList[i][str(yr)+' Median Rental Price'] = rentCleanData[str(yr)+' Median Rental Price']
    yearData = pd.DataFrame(ZdataList[i])
    ZZdataList.append(yearData)
    i = i + 1
    yr = yr + 1
    
ZZdataList[23].head()

C:\Users\denni\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\denni\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,Zipcode,2019 Median Home Value,2019 Median Rental Price
0,10025,995443.0,3422.0
2,10023,1379100.0,3614.0
8,10002,927014.0,3144.0
9,10467,428600.0,1505.0
10,11226,825086.0,1659.0
